In [ ]:
%load_ext autoreload
%autoreload 2

Description:

Ce notebook est utile pour afficher les courbes de tendance de SoH par modèle (pour l'instant tesla) et on peut même aller plus loin avec les courbe de tendance de SoH pour les versions (ou tesla code).

Il y'a la possibilité d'ajouter les intervals de confiance de ces courbes. 

In [ ]:
import pandas as pd
from core.sql_utils import *
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import curve_fit
import numpy as np
import matplotlib.pyplot as plt

from transform.insights_results.trendlines import *


In [ ]:
engine = get_sqlalchemy_engine()
con = engine.connect()

with engine.connect() as connection:
    dbeaver_df = pd.read_sql(text("""SELECT *  FROM  vehicle v	
            join vehicle_model vm 
            on vm.id = v.vehicle_model_id
            join vehicle_data vd 
            on vd.vehicle_id  = v.id
            join oem o 
            on o.id = vm.oem_id
            join battery b 
            on b.id = vm.battery_id""")
            #where o.id in ('5b89ebf7-8774-41ec-97f8-7dd4e8a2fd0f');""")
                             , con)



dbeaver_df.head()
dbeaver_df['timestamp'] = pd.to_datetime(dbeaver_df['timestamp'])
dbeaver_df.sort_values("timestamp", inplace=True)

In [ ]:
dbeaver_df.columns

In [ ]:
# on drop MTY13/MT336/MT337
dbeaver_filtered = dbeaver_df[~dbeaver_df['version'].isin(['MT336','MT337'])][['version', 'soh', 'odometer', 'model_name', 'type', 'vin', 'oem_name']].copy().dropna()


In [ ]:
px.scatter(dbeaver_filtered[['version', 'soh', 'odometer']],  x='odometer', y='soh', color="version")

## Graphs des tendances 

In [ ]:
def trendline_apply(x, f):
    return eval(f)
    
    

In [ ]:
dbeaver_filtered

In [ ]:
import plotly.graph_objects as go

for marque in ["tesla", 'renault', "mercedes", "ford", "volvo"]:
    df_marque = dbeaver_filtered[dbeaver_filtered['oem_name'] == marque]
    trendline, trendline_max, trendline_min = get_trendlines(df_marque)
    
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=df_marque['odometer'],
        y=df_marque['soh'],
        mode='markers',
        marker_color='rgba(50, 182, 193, .9)',
        name='SoH compute'
    ))

    x_sorted = df_marque['odometer'].sort_values()
    print()
    fig.add_trace(go.Scatter(
        x=x_sorted,
        y=trendline_apply(x_sorted, trendline['trendline']),
        mode='lines',
        line=dict(color='red'),
        name='Fit'
    ))

    fig.add_trace(go.Scatter(
        x=x_sorted,
        y=trendline_apply(x_sorted, trendline_max['trendline']),
        mode='lines',
        line=dict(color='green'),
        name='Upper'
    ))

    fig.add_trace(go.Scatter(
        x=x_sorted,
        y=trendline_apply(x_sorted, trendline_min['trendline']),
        mode='lines',
        line=dict(color='green'),
        name='Lower'
    ))

    fig.update_layout(
        width=1000,
        height=600,
        xaxis_title='Odometer',
        yaxis_title='State of Health (SoH)',
        legend_title='Légende',
        title=f"Marque: {marque}",
        template='plotly_white'
    )

    fig.show()

### GSheet

In [ ]:
import gspread
from google.oauth2.service_account import Credentials

# <!-- # Charger les identifiants
# creds = Credentials.from_service_account_file(
#     'chemin/vers/le/fichier.json',
#     scopes=["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"] -->
# )

In [ ]:
creds = Credentials.from_service_account_file(
    '../../ingestion/vehicle_info/config/config.json',
    scopes=["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
)

In [ ]:
client = gspread.authorize(creds)


In [ ]:
sheet = client.open("202505 - Courbes SoH")  


In [ ]:
worksheets = sheet.worksheets()


In [ ]:
worksheets

In [ ]:
coubes_sheet = sheet.worksheet("Courbes OS")


In [ ]:
df_spreed_sheet = pd.DataFrame(coubes_sheet.get_all_values()[1:], columns=coubes_sheet.get_all_values()[0])

In [ ]:
df_spreed_sheet["Odomètre (km)"] = df_spreed_sheet["Odomètre (km)"].apply(lambda x: x.replace(" ", "")).astype(int)
df_spreed_sheet["SoH"] = df_spreed_sheet["SoH"].apply(lambda x: x.replace("%", "")).astype(float)

In [ ]:
px.scatter(df_spreed_sheet, x='Odomètre (km)', y='SoH', color='OEM')